In [328]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder

from category_encoders import MEstimateEncoder
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import cross_val_score
# from category_encoders import TargetEncoder

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, Lasso, LassoCV, LassoLarsCV
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Preprocessing

I will begin by using the knowledge I gained in my EDA to remove outliers and drop irrelevant columns.

Once my base models were built, I reintroduced other variables back in. I added them back in from highest Mutual Information/Correlation downwards.

In [377]:
df_all = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv', index_col=0)

# Feature engineering: total bathrooms
df_all['TotalBath'] = df_all['BsmtFullBath'] + df_all['FullBath'] + 0.5*df_all['BsmtHalfBath'] + 0.5*df_all['HalfBath']

df_dropped = df_all.drop(['PoolQC', 'MiscFeature', 'Alley'], axis=1)
df = df_dropped[[
       'OverallQual', 
       'Neighborhood', 
       'GarageArea', 
       'GrLivArea', 
       'YearBuilt',
       'TotalBsmtSF', 
       'LotArea', 
       'BsmtQual', 
       'ExterQual',
       'KitchenQual', 
       '1stFlrSF', 
       'MSSubClass', 
       'YearRemodAdd',
       'FullBath',
       'GarageFinish', 
       'GarageYrBlt', 
       'LotFrontage', 
       'FireplaceQu',
       'TotRmsAbvGrd', 

       'TotalBath',
       'Foundation',
       'GarageType',
       'OpenPorchSF',
       'HeatingQC',
       'Fireplaces',
       'MSZoning',
       'OverallCond',
       'MasVnrType',
       'WoodDeckSF',
       # 'LotShape',
       'MasVnrArea',
       # 'HouseStyle',
       # 'GarageCond',
       # 'PavedDrive',

       'SalePrice'
       ]].copy()
df = df.drop(df[(df['SalePrice']<300000) & (df['GrLivArea'] > 4000)].index)


y = df.pop('SalePrice')
log_y = np.log(y)

## Pipeline

One could argue there is little need for a pipeline with a relatively simple model, however it is good pratice beacuse it reduces data leakage in cross validation and allows for easy experimentation with models and preprocessing methods.

Remember a column transformer can be used to transform different columns but it doesn't link the transforms togther. So if you need to apply to transformations to one column you need to put them in another pipeline then put that pipeline in the column transformer, as I've done here.

In [325]:
# Need to just use multivariate imputer for this I think
# def custom_imputer(df):
#     df['LotFrontage'] = df.groupby(by='Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
#     return df
# custom_imputer_transformer = FunctionTransformer(func=custom_imputer, validate=False)

def log_scaler(df):
    df['GrLivArea'] = np.log(df['GrLivArea'])
    return df

log_transformer = FunctionTransformer(func=log_scaler, validate=False)

# Orindal encoding setup
ordinal_features = [
                    'GarageFinish',
                    'BsmtQual',
                    'ExterQual',
                    'KitchenQual', 
                    'FireplaceQu',
                    'HeatingQC',
                    ]

five_lvls = ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
garage_lvls = ['None', 'Unf', 'RFn', 'Fin']
drive_lvls = ['N','P','Y']
all_lvls = [garage_lvls,
            five_lvls,
            five_lvls,
            five_lvls,
            five_lvls,
            five_lvls,
]


# Target encoding
target_enc_features = ['Neighborhood']

# Onehot encoding setup
onehot_features = ['MSSubClass','Neighborhood','Foundation','MSZoning']

# Features to log scale
log_features = ['GrLivArea']

# Features that are missing and need onehot enc
missing_onehot = ['GarageType', "MasVnrType"]

features_impute_zero = ['GarageYrBlt','OpenPorchSF','Fireplaces','WoodDeckSF','MasVnrArea']

preprocessPipe1 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='None')),
    ('encoder', OrdinalEncoder(categories=all_lvls, 
                                        handle_unknown='error',
    ))
])

preprocessPipe2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='None')),
    ('OH', OneHotEncoder(handle_unknown='ignore'))
])


# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('Pipe1', preprocessPipe1, ordinal_features),
        ('Pipe2', preprocessPipe2, missing_onehot),
        ('MedianImputer', SimpleImputer(strategy='median'), ['LotFrontage']),
        ('imputeZero',  SimpleImputer(strategy='constant', fill_value=0), features_impute_zero),
        ('log_scaler', log_transformer, log_features),
        ('imputeCond',SimpleImputer(strategy='constant', fill_value=5),['OverallCond']),
        # ('targetEncoder', MEstimateEncoder( m=.06), target_enc_features),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), onehot_features),
        
    ], remainder='passthrough')


Checking everything is working with a quick cheeky Cross-validation

In [323]:
# Create the final pipeline with the preprocessor and your model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=230, random_state=1))
])

def cv(my_pipeline):
    scores = -1 * cross_val_score(my_pipeline, df, log_y,
                                  cv=5,
                                  scoring='neg_mean_squared_error')
    return np.mean(np.sqrt(scores))

print(f'The RMSLE in SalePrice is : \n{cv(pipeline)}')

The RMSLE in SalePrice is : 
0.13990342233193753


## Grid search

First, I will now look at a few linear regression models (including regularised models such as Ridge and Lasso), running a grid search on their parameters.

I will then go onto train ensemble models, such as Random Forest, XGBoost.

I first looked at implementing the hyperparameterisation with Optuna, but decided this was unnecessariily complex and made my code unreadable. I have gone back to using simple GridSearchCV

There may be a way to integrate this individual model search
    into GridSearchCV however because each modle has a different param
    set I proceeded this way to make it simpler


In [431]:
def run_search(model, params, tracker):

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    gs= GridSearchCV(pipeline,
                    param_grid=params,
                    scoring='neg_mean_squared_error',
                    cv=5,
                    verbose=1
                )
    
    gs.fit(df,log_y);

    # scores = np.array([-1*gs.cv_results_[f'split{i}_test_score'] for i in range(gs.n_splits_)])
    # best_RMSLE = min(np.sqrt(np.mean(scores,axis=0)))

    mean_score = -1 * gs.cv_results_['mean_test_score']
    best_RMSLE = min(np.sqrt(mean_score))

    
    tracker[str(model)] = best_RMSLE
    
    print(f'For {str(model)}, optimal param when {gs.best_params_} with score RMSLE: \n{best_RMSLE}')

    return gs


In [ ]:
def plot_grid_search(cv_results, grid_param_1, grid_param_2, name_param_1, name_param_2):
    # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_score']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_2),len(grid_param_1))

    scores_sd = cv_results['std_test_score']
    scores_sd = np.array(scores_sd).reshape(len(grid_param_2),len(grid_param_1))

    # Plot Grid search scores
    _, ax = plt.subplots(1,1)

    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        ax.plot(grid_param_1, scores_mean[idx,:], '-o', label= name_param_2 + ': ' + str(val))

    ax.set_title("Grid Search Scores", fontsize=20, fontweight='bold')
    ax.set_xlabel(name_param_1, fontsize=16)
    ax.set_ylabel('CV Average Score', fontsize=16)
    ax.legend(loc="best", fontsize=15)
    ax.grid('on')


### Linear models

In [432]:
tracker = {}

model = Ridge(alpha=0.0001)
params = {'model__alpha' : list(np.arange(0.001,4,0.01))}
ridge = run_search(model,params,tracker)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits
For Ridge(alpha=0.0001), optimal param when {'model__alpha': 2.6909999999999994} with score RMSLE: 
0.11880181562320062


In [436]:
ridge.best_params_

{'model__alpha': 2.6909999999999994}

In [437]:
model = Lasso(alpha=0.0001)
params = {'model__alpha' : list(np.arange(0.0001,0.001,0.00001))}
lasso = run_search(model,params,tracker)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


/Users/edroberts/opt/anaconda3/envs/trading/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.237e-02, tolerance: 1.800e-02
  model = cd_fast.enet_coordinate_descent(
/Users/edroberts/opt/anaconda3/envs/trading/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.494e-02, tolerance: 1.800e-02
  model = cd_fast.enet_coordinate_descent(
/Users/edroberts/opt/anaconda3/envs/trading/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

For Lasso(alpha=0.0001), optimal param when {'model__alpha': 0.00023999999999999998} with score RMSLE: 
0.11806632789247537


In [438]:
model = ElasticNet(alpha=0.0001, l1_ratio=1.0)
params = {'model__alpha' : list(np.arange(0.0001,0.001,0.00001)),
          'model__l1_ratio': [1.0, 0.9, 0.8, 0.7]}
elastic = run_search(model,params,tracker)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/Users/edroberts/opt/anaconda3/envs/trading/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.237e-02, tolerance: 1.800e-02
  model = cd_fast.enet_coordinate_descent(
/Users/edroberts/opt/anaconda3/envs/trading/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.106e+00, tolerance: 1.800e-02
  model = cd_fast.enet_coordinate_descent(
/Users/edroberts/opt/anaconda3/envs/trading/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

For ElasticNet(alpha=0.0001, l1_ratio=1.0), optimal param when {'model__alpha': 0.00023999999999999998, 'model__l1_ratio': 1.0} with score RMSLE: 
0.11806632789247537


In [439]:
tracker

{'Ridge(alpha=0.0001)': 0.11880181562320062,
 'Lasso(alpha=0.0001)': 0.11806632789247537,
 'ElasticNet(alpha=0.0001, l1_ratio=1.0)': 0.11806632789247537}

### Ensemble time

In [440]:
model = RandomForestRegressor()
params = {'model__n_estimators': range(30,300,10),
          'model__random_state': [1]}
run_search(model,params,tracker)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
For RandomForestRegressor(), optimal param when {'model__n_estimators': 290, 'model__random_state': 1} with score RMSLE: 
0.13930426497249512


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Pipe1',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='None',
                                                                                                        strategy='constant')),
                                                                                         ('encoder',
                                                                                          OrdinalEncoder(categories=[['None',
                                                                                                                      'Unf',
                                                                                                                      'RFn',
                                                                                                                      'Fin'],
                                                                                                                     ['None',
                                                                                                                      'Po',
                                                                                                                      'Fa',
                                                                                                                      'TA',
                                                                                                                      'Gd',
                                                                                                                      'Ex'],
                                                                                                                     ['None',
                                                                                                                      'Po',
                                                                                                                      'Fa',
                                                                                                                      'TA',
                                                                                                                      'Gd',
                                                                                                                      'Ex'],
                                                                                                                     ['None',
                                                                                                                      'Po...
                                                                         ['GrLivArea']),
                                                                        ('imputeCond',
                                                                         SimpleImputer(fill_value=5,
                                                                                       strategy='constant'),
                                                                         ['OverallCond']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['MSSubClass',
                                                                          'Neighborhood',
                                                                          'Foundation',
                                                                          'MSZoning'])])),
                               

In [441]:
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
params = {'model__n_estimators': range(500,2000,500),
          'model__learning_rate' : list(np.arange(0.01,0.1,0.01))
          }
run_search(model,params,tracker)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


For XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...), optimal param when {'model__learning_rate': 0.02, 'model__n_estimators': 1000} with score RMSLE: 
0.12834117677854234


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Pipe1',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='None',
                                                                                                        strategy='constant')),
                                                                                         ('encoder',
                                                                                          OrdinalEncoder(categories=[['None',
                                                                                                                      'Unf',
                                                                                                                      'RFn',
                                                                                                                      'Fin'],
                                                                                                                     ['None',
                                                                                                                      'Po',
                                                                                                                      'Fa',
                                                                                                                      'TA',
                                                                                                                      'Gd',
                                                                                                                      'Ex'],
                                                                                                                     ['None',
                                                                                                                      'Po',
                                                                                                                      'Fa',
                                                                                                                      'TA',
                                                                                                                      'Gd',
                                                                                                                      'Ex'],
                                                                                                                     ['None',
                                                                                                                      'Po...
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=1000,
                                                     n_jobs=4,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             param_grid={'model__learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05,
                                                  0.06, 0.07, 0.08, 0.09],
                         'model__n_estimators': range(500, 2000, 500)},
             scoring='neg_mean_squared_error', verbose=1)

In [442]:
tracker

{'Ridge(alpha=0.0001)': 0.11880181562320062,
 'Lasso(alpha=0.0001)': 0.11806632789247537,
 'ElasticNet(alpha=0.0001, l1_ratio=1.0)': 0.11806632789247537,
 'RandomForestRegressor()': 0.13930426497249512,
 'XGBRegressor(base_score=None, booster=None, callbacks=None,\n             colsample_bylevel=None, colsample_bynode=None,\n             colsample_bytree=None, device=None, early_stopping_rounds=None,\n             enable_categorical=False, eval_metric=None, feature_types=None,\n             gamma=None, grow_policy=None, importance_type=None,\n             interaction_constraints=None, learning_rate=0.05, max_bin=None,\n             max_cat_threshold=None, max_cat_to_onehot=None,\n             max_delta_step=None, max_depth=None, max_leaves=None,\n             min_child_weight=None, missing=nan, monotone_constraints=None,\n             multi_strategy=None, n_estimators=1000, n_jobs=4,\n             num_parallel_tree=None, random_state=None, ...)': 0.12834117677854234}

## Stacking models

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    '''Taken from:
    https://www.kaggle.com/code/serigne/stacked-regressions-top-4-on-leaderboard#Stacked-Regressions-to-predict-House-Prices
    '''

    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1) 

In [499]:
x = 0.92
average_prediction = (ridge.predict(df)*x + (1-x)*lasso.predict(df) + (1-x)*elastic.predict(df)) / (2-x)

In [501]:
preds = pd.DataFrame({'log_y':log_y, 'log_preds' : average_prediction},
                     index = df.index.copy(),
)
preds
np.sqrt(np.mean((preds['log_y'] - preds['log_preds'])**2))

0.1114568862990213

In [462]:
np.sqrt(np.mean((preds['log_y'] - preds['log_preds'])**2))

0.11147542019525089

In [ ]:
averaged_models = AveragingModels(models = (elastic, ridge, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

## Making prediction to submit